# Futures sum

Using dask futures API

In [ ]:
import cudf
from numba import cuda

In [ ]:
import dask
import distributed
import cudf
import pandas as pd
import os
import numpy as np

In [ ]:
print(f'Dask: {dask.__file__}')
print(f'Distributed: {distributed.__file__}')
print(f'cuDF: {cudf.__file__}')

In [ ]:
base_env = {
    "UCX_RNDV_SCHEME": "put_zcopy",
    "UCX_MEMTYPE_CACHE": "n",
    "UCX_TLS": "rc,cuda_copy,cuda_ipc",
    "CUDA_VISIBLE_DEVICES": "2,3",
}
os.environ.update(base_env)

In [ ]:
from dask.distributed import Client, wait
from dask_cuda import DGX

cluster = DGX(CUDA_VISIBLE_DEVICES=[0,1], 
              dashboard_address='10.33.227.165:8789')
client = Client(cluster)
# client = Client('ucx://10.33.225.165:42841')
client

## columns

In [ ]:
from dask.distributed import wait
import operator

left = client.map(lambda x : cudf.Series([x])._column, range(1000), workers=[0])
right = client.map(lambda x : cudf.Series([x])._column, range(1000), workers=[1])
results = client.map(lambda x,y: (x,y), left, right)
wait(results)

## series

In [ ]:
from dask.distributed import wait
import operator

left = client.map(lambda x: cudf.Series([x]), range(1000), workers=[0])
right = client.map(lambda x: cudf.Series([x]), range(1000), workers=[1])
results = client.map(lambda x,y:(x,y), left, right)
wait(results)